In [1]:
import torch

ModuleNotFoundError: No module named 'torch'

In [2]:
print("CUDA available:", torch.cuda.is_available())
print("GPU:",torch.cuda.get_device_name(0) if torch.cuda.is_available() else "No GPU")

CUDA available: True
GPU: NVIDIA GeForce RTX 3050 Laptop GPU


# load dataset from the hugging face

In [4]:
from datasets import load_dataset

data = load_dataset("imdb")

/home/breezy-s-pc/Study/student_feedback_sentiment_analysis/venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
data

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    unsupervised: Dataset({
        features: ['text', 'label'],
        num_rows: 50000
    })
})

In [6]:
from transformers import BertTokenizer

In [7]:
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

In [8]:
def tokenize_function(examples):
    return tokenizer(examples["text"],padding ="max_length",truncation=True)

# Apply the torkernizer for function to the dataset

In [9]:
tokenized_data = data.map(tokenize_function)
print(tokenized_data)

DatasetDict({
    train: Dataset({
        features: ['text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 25000
    })
    test: Dataset({
        features: ['text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 25000
    })
    unsupervised: Dataset({
        features: ['text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 50000
    })
})


In [10]:
train_testvalid = tokenized_data["train"].train_test_split(test_size=0.2)
train_dataset = train_testvalid["train"]
valid_dataset = train_testvalid["test"]


In [11]:
type(train_testvalid)

datasets.dataset_dict.DatasetDict

# load the dataloader to batch the data (splitt the data chunks )

In [12]:
from torch.utils.data import DataLoader

In [13]:
train_dataloader = DataLoader(train_dataset, batch_size=8, shuffle=True)
valid_dataloader = DataLoader(valid_dataset, batch_size=8,)

# fine tune the bertmodel using my data

In [14]:
from transformers import BertForSequenceClassification, TrainingArguments, Trainer

KeyboardInterrupt: 